# Floyd-Warchall

## Algorithm and experiment functions

In [6]:
using BenchmarkTools
using Statistics
using Random

# Graph generating function
function generate_graph_fw(n::Int, edge_prob::Float64 = 0.5)
    A = fill(Inf, n, n)
    for i in 1:n
        A[i,i] = 0.0
        for j in 1:n
            if j != i && rand() < edge_prob
                # Allow negative weights and avoid negative self-loops
                A[i,j] = rand(-5.0:1.0:10.0) 
            end
        end
    end

    return A
end

# Main Algorithm Floyd Warshall
function floydwarshall(A::Matrix{Float64})
    n = size(A, 1)
    dist = copy(A)

    for k in 1:n
        for i in 1:n
            for j in 1:n
                if dist[i, j] > dist[i, k] + dist[k, j]
                    dist[i, j] = dist[i, k] + dist[k, j]
                end
            end
        end
    end

    return dist
end

# Function to check for negative cycles
function has_negative_cycle(A::Matrix{Float64})
    for i in 1:size(A, 1)
        if A[i, i] < 0.0
            return true
        end
    end
    return false
end

# Time estimator for later experiments
function estimate_instances_fw(A::Matrix{Float64}, target_time::Float64 = 420.0)
    test_instances = 3
    instance_times = [@elapsed floydwarshall(copy(A)) for _ in 1:test_instances]
    avg_time = mean(instance_times)

    estimated_instances = max(1, Int(round(target_time / avg_time)))

    return estimated_instances
end

# Time measuring function
function measure_fw_time(A::Matrix{Float64}, target_time::Float64 = 420.0)
    instances = estimate_instances_fw(A, target_time)
    negative_cycle_count = 0
    instance_times = Vector{Float64}()
    for _ in 1:instances
        A_copy = deepcopy(A)
        dist = nothing
        time = @elapsed begin 
            dist = floydwarshall(A_copy)
        end
        push!(instance_times, time)

        if has_negative_cycle(dist)
            negative_cycle_count += 1
        end

    end
    println("Negative cycles found: $negative_cycle_count for size $(size(A, 1))")

    avg_time = mean(instance_times)
    median_time = median(instance_times)
    total_time = sum(instance_times)
    
    return avg_time, median_time, total_time, instances, instance_times
end

# Function for runnig whole experiment using additional functions
function run_experiment(edge_prob::Float64 = 0.8, target_time::Float64 = 420.0)
    sizes = [50, 100, 200, 400, 600, 800, 1000, 1200]
    all_times = Vector{Vector{Float64}}() 

    for size in sizes
        A = generate_graph_fw(size, edge_prob)
        avg_time, median_time, total_time, instances, instance_times = measure_fw_time(A, target_time)
        push!(all_times, instance_times)
        println("Size: $size | Instances: $instances | Avg Time: $avg_time | Median Time: $median_time | Total Time: $total_time")
        println("--------------------------------------------------")
    end

    return all_times
end

run_experiment (generic function with 3 methods)

## Run experiment and make violin plot

In [7]:
using StatsPlots
using Dates

# Run experiment
times = run_experiment(0.75, 420.0)
sizes_series = ["50" "100" "200" "400" "600" "800" "1000" "1200"]

violin(
    sizes_series,
    times,
    xlabel = "Sizes (n)",
    ylabel = "Times (s)",
    title = "Floyd-Warshall Algorithm Performance",
    show_median = true,
    legend = false,
    yscale = :log10,
)

savefig("plots/floyd_warshall_violin_plot_$(Dates.format(Dates.now(), "yyyymmdd_HHMM")).png")

"c:\\VSCodeProjects\\Julia\\path-finding\\plots\\floyd_warshall_violin_plot_20250419_0032.png"